![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# United Nations



### Code: 

Run the code cells below to import the libraries we need for this project. Libraries are pre-made code that make it easier to analyze our data.

In [ ]:
import pandas as pd
import plotly_express as px
import folium
import geopandas as gpd
import plotly.graph_objs as go
from ipywidgets import interact
print("Libraries imported.")

In [ ]:
HDI_components = pd.read_excel("https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/united-nations/HDI_components.xlsx")
HDI_inequality = pd.read_excel("https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/united-nations/HDI_inequality_adjusted.xlsx")
file_path = "https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/united-nations/HDI_multidimensional_poverty.xlsx"
sheet1_name = 'table1'
sheet2_name = 'table2'
HDI_multidimensional_poverty = pd.read_excel(file_path, sheet_name=sheet1_name)
HDI_multidimensional_poverty2 = pd.read_excel(file_path, sheet_name=sheet2_name)
HDI_years = pd.read_excel("https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/united-nations/HDI_years.xlsx")

display(HDI_components)
display(HDI_inequality)
display(HDI_multidimensional_poverty)
display(HDI_multidimensional_poverty2)
display(HDI_years)

### Examining HDI Trends

In [ ]:
HDI_years

In [ ]:
HDI_cols = HDI_years.columns[2:10]
for column in HDI_cols:
    HDI_years[column] = pd.to_numeric(HDI_years[column], errors='coerce')
    print(f"Maximum value in {column}: {HDI_years[column].max()}")
    max_hdi_country = HDI_years.loc[HDI_years[column].idxmax()]['Country']
    print(f"Country with the highest HDI: {max_hdi_country}")

In [ ]:
def update_scatter_plot(selected_year):
    data = []
    for country in HDI_years['Country']:
        trace = go.Scatter(x=[country], y=[HDI_years.loc[HDI_years['Country'] == country, selected_year].values[0]], mode='markers', name=country)
        data.append(trace)

    layout = go.Layout(title=f'HDI Scatter Plot ({selected_year})', xaxis=dict(showticklabels=False, title='Country'), yaxis=dict(title='HDI'))
    selected_year_fig = go.Figure(data=data, layout=layout)
    selected_year_fig.show()

interact(update_scatter_plot, selected_year=HDI_cols)

In [ ]:
countries_geojson = gpd.read_file('countries.geojson')
countries_geojson

In [ ]:
no_HDI_years = HDI_years[HDI_years['HDI rank'].isnull()]
no_HDI_years

In [ ]:
HDI_components

In [ ]:
countries = HDI_components.Country.unique()
for country in countries:
    print(country)

In [ ]:
no_HDI = HDI_components[HDI_components['HDI rank'].isnull()]
no_HDI